# 训练教程


## Install Dependencies

In [ ]:
!pip install transformers==4.28.1
!pip install git+https://github.com/huggingface/peft.git@13e53fc
!pip install datasets
!pip install sentencepiece
!pip install deepspeed

## 克隆代码





In [ ]:
!git clone https://github.com/IFOSR/BuffettGPT.git

## 指令微调 Alpaca-7B

参考chinese llama的博客 https://github.com/ymcui/Chinese-LLaMA-Alpaca/wiki/SFT-Script 设置对应的参数

In [ ]:
# 对应目录记得改成自己的

!cd [脚本地址]/train && torchrun --nnodes 1 --nproc_per_node 1 run_clm_sft_with_peft.py \
    --deepspeed ds_zero2_no_offload.json \
    --model_name_or_path decapoda-research/llama-7b-hf \
    --tokenizer_name_or_path ziqingyang/chinese-alpaca-lora-7b \
    --dataset_dir [数据地址]/data \
    --validation_split_percentage 0.001 \
    --per_device_train_batch_size 1 \
    --do_train \
    --fp16 \
    --seed $RANDOM \
    --max_steps 10000 \
    --lr_scheduler_type cosine \
    --learning_rate 1e-4 \
    --warmup_ratio 0.03 \
    --weight_decay 0 \
    --logging_strategy steps \
    --logging_steps 10000 \
    --save_strategy steps \
    --save_total_limit 2 \
    --save_steps 10000 \
    --gradient_accumulation_steps 1 \
    --preprocessing_num_workers 8 \
    --max_seq_length 512 \
    --output_dir [输出地址] \
    --overwrite_output_dir \
    --ddp_timeout 30000 \
    --logging_first_step True \
    --torch_dtype float16 \
    --peft_path [peft 模型的地址] \
    --gradient_checkpointing \
    --ddp_find_unused_parameters False

# 合并模型
参考Chinese llama的文档：https://github.com/ymcui/Chinese-LLaMA-Alpaca/wiki/%E5%9C%A8%E7%BA%BF%E6%A8%A1%E5%9E%8B%E5%90%88%E5%B9%B6%E4%B8%8E%E8%BD%AC%E6%8D%A2

In [ ]:
!python merge/merge_llama_with_chinese_lora.py \
    --base_model  decapoda-research/llama-7b-hf\
    --lora_model ziqingyang/chinese-llama-plus-lora-7b,[你训练完的模型的地址]/sft_lora_model \
    --output_type pth \
    --output_dir llama-combined-ch

# llama.cpp量化部署


## 克隆代码和编译

In [ ]:
!git clone https://github.com/ggerganov/llama.cpp

In [ ]:

!cd llama.cpp && make

## 模型转换为ggml格式（FP16）

In [ ]:
!cd llama.cpp && python convert.py llama-combined-ch

## 测试模型

In [ ]:
!cd llama.cpp && chmod 777 main && ./main -m llama-combined-ch/ggml-model-f16.bin --color -f ./prompts/alpaca.txt -ins -c 2048 --temp 0.2 -n 256 --repeat_penalty 1.1

## 如果想量化模型请参考
https://github.com/ymcui/Chinese-LLaMA-Alpaca/blob/main/notebooks/convert_and_quantize_chinese_llama_and_alpaca.ipynb